In [1]:
import os
import pandas as pd  # type: ignore
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM, get_scheduler, TrainingArguments
from datasets import load_dataset, Dataset
from peft import PeftModel, LoraConfig, get_peft_model
from trl import SFTTrainer, setup_chat_format, SFTConfig
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
device = torch.device("mps")

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [5]:
df = pd.read_xml("./data/dataset.xml")
del df['index']
df.head()

,input,output
0,What is a 'customized integrated framework' in...,In the context of Audit and Assurance policy a...
1,What are some of the elements that audit and a...,Audit and assurance policies and procedures sh...
2,Why is it important to review and update the a...,It's important to review and update the audit ...
3,What is the role of an audit report in the aud...,An audit report plays a crucial role in the au...
4,What is the purpose of follow-up activities in...,Follow-up activities are designed to monitor t...


In [6]:
dataset = Dataset.from_pandas(df)

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["input"]},
               {"role": "assistant", "content": row["output"]}]
    row["text"] = tokenizer.apply_chat_template(
        row_json,
        add_generation_prompt=True,
        include_special_tokens=True,
        tokenize=False
    )
    return row

dataset = dataset.map(
    format_chat_template,
)


Map:   0%|          | 0/1038 [00:00<?, ? examples/s]Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using cls_token, but it 

In [7]:
dataset = dataset.train_test_split(test_size=0.1)

In [8]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.14s/it]


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model, tokenizer = setup_chat_format(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
sft_config = SFTConfig(
    output_dir="ccm_lora_weights",
    dataset_text_field="text",
    num_train_epochs=1,
    max_seq_length=512,
    learning_rate=2e-4,
)

In [11]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    peft_config=peft_config,
    args=sft_config,
)
trainer.train()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Using pad_token, but it is not set yet.
Map: 100%|██████████| 934/934 [00:00<00:00, 20116.26 examples/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable c

 14%|█▎        | 16/117 [34:11<3:50:25, 136.88s/it]Python(11521) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11524) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11528) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11534) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11538) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11542) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11544) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11549) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11552) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(11555) MallocStackLogging: can't turn off malloc stack loggi

In [ ]:
model.save_pretrained("ccm_lora_weights")